# Relevant News For Netherlands Expats

A solution for having up-to-date information relevant to expats in the Netherlands. It scarps main news portals and makes a summary with relevant information.

## List of Sources
Defined by the user

## Setup

In [ ]:
#imports

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import gradio as gr

In [13]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_LINKS = 'gpt-5-nano'      #small/cheap model for link selection
MODEL_DIGEST = 'gpt-4.1-mini'   #better model for final digest
openai = OpenAI()

DEFAULT_MAX_CATEGORIES = 3
DEFAULT_MAX_ARTICLES_PER_CATEGORY = 5
DEFAULT_MAX_ARTICLES_TOTAL = 15

API key looks good so far


## Prompts

In [3]:
category_link_system_prompt = """
You are provided with a list of links found on the homepage of a Netherlands news website.
Your task is to pick links that are *news category/section* pages (e.g. politics, economy, housing, health, migration, transport, culture).
Exclude: privacy/terms, about, contact, subscribe, login, donate, ads, newsletters, social media, PDFs, mailto links.

Return JSON in this exact shape:

{
  "categories": [
    {"name": "politics", "url": "https://full.url/to/category"},
    {"name": "economy", "url": "https://full.url/to/category"}
  ]
}

Rules:
- URLs must be full absolute https URLs.
- Prefer a diverse set of categories (not 6 variations of the same thing).
- If the site is in Dutch, still return category names in English.
"""

article_link_system_prompt = """
You are provided with a list of links found on a *category page* of a Netherlands news website.
Pick links that are *individual news articles* from that category.

Return UP TO {max_articles} articles. If there are enough, return as close to {max_articles} as possible.
Prefer the most recent-looking ones. Always include housing market news.
Exclude: category index pages, tag pages, author pages, pagination, subscribe/login, privacy/terms, newsletters, social media.

Return JSON in this exact shape:

{
  "articles": [
    {"title": "short title", "url": "https://full.url/to/article"},
    {"title": "short title", "url": "https://full.url/to/article"}
  ]
}

Rules:
- URLs must be full absolute https URLs.
- Titles should be short; if the title isn't obvious from the URL, make a best guess.
"""

digest_system_prompt = """
You are an helpfull assistant creating a practical news digest for expats living in the Netherlands. Your ton of voice a bit sarcastic and straightforward and can remind Mark Manson style.
You will be given scraped text from several recent news articles.

Write in English, in markdown WITHOUT code blocks.

Requirements:
- Start with a short "What were important events this week" section (3-7 bullets).
- Then group items by theme 5-7 for each gourp (e.g. migration/IND, taxes/benefits, housing, transport, healthcare, safety, education, politics/economy).
- For each item: 1-2 sentence summary + "why it's important for expats" + (optional) "what you can do" if there is a practical action.
- At the end of each group include "Sources" as a bullet list of URLs used.
- If the scraped text is incomplete (paywall/JS), say so and avoid overconfident claims.
"""

## Step 1: category selection

In [ ]:
def get_category_links_user_prompt(url):
    links = fetch_website_links(url)

    user_prompt = f"""
Here is the list of links on the website {url} -
Pick the category/section pages.

Links (some might be relative links):
"""
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
def selec_category_links(url, max_categories=DEFAULT_MAX_CATEGORIES):
    print(f"Selecting relevant category links for {url} by calling {MODEL_LINKS}")
    response = openai.chat.completions.create(
        model=MODEL_LINKS,
        messages=[
            {"role": "system", "content": category_link_system_prompt},
            {"role": "user", "content": get_category_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    
    data = json.loads(response.choices[0].message.content)

    categories = data.get("categories", [])
    if not isinstance(categories, list):
        categories = []

    print(f"Found {len(categories)} relevant category links")

    data["categories"] = categories[:max_categories]
    return data

## Step 2: article selection per category

In [ ]:

def get_article_links_user_prompt(category_url, max_links=300):
    links = fetch_website_links(category_url)

    links = [str(l).strip() for l in links if l]

    links = list[str](dict.fromkeys(links))
    links = links[:max_links]
    user_prompt = f"""
Here is the list of links found on the category page {category_url}

Pick recent article links. 
Links (some might be relative links):
"""
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
def select_recent_articles(category_url, max_articles=DEFAULT_MAX_ARTICLES_PER_CATEGORY):
    response = openai.chat.completions.create(
        model=MODEL_LINKS,
        messages=[
            {"role": "system", "content": article_link_system_prompt},
            {"role": "user", "content": get_article_links_user_prompt(category_url)},
        ],
        response_format={"type": "json_object"},
    )
    data = json.loads(response.choices[0].message.content)
    articles = data.get("articles", [])

    result = []
    for a in articles:
        url = a.get("url")
        title = a.get("title")
        if not url:
            continue
        result.append({"title": title, "url": url})
    print(f"Selected {len(result)} articles")
    return result[:max_articles]

## Step 3: fetch + aggregate text

In [8]:
def fetch_homepage_categories_and_articles(
    home_url,
    max_categories=DEFAULT_MAX_CATEGORIES,
    max_articles_per_category=DEFAULT_MAX_ARTICLES_PER_CATEGORY,
    max_articles_total=DEFAULT_MAX_ARTICLES_TOTAL,
):

    homepage = fetch_website_contents(home_url)
    categories_data = selec_category_links(home_url, max_categories=max_categories)
    categories = categories_data.get("categories", [])

    #Pick articles
    all_articles = []
    for cat in categories:
        cat_url = cat.get("url")
        cat_name = cat.get("name", "unknown")

        picked = select_recent_articles(cat_url, max_articles=max_articles_per_category)
        for a in picked:
            all_articles.append({"category": cat_name, **a})
        if len(all_articles) >= max_articles_total:
            break
    all_articles = all_articles[:max_articles_total]

# Fetch articles contents
    result = f"## Homepage ({home_url})\n\n{homepage}\n\n## Articles\n"
    for a in all_articles:
        result += f"\n\n###Category: {a['category']}\n"
        result += f"### Title (from link selection): {a['title']}\n"
        result += f"### URL: {a['url']}\n\n"
        result += fetch_website_contents(a["url"])

# Keep input reasonably bounded
    print(f"Fetching data")
    return result[:18_000], [a["url"] for a in all_articles]

def get_digest_user_prompt(home_url, **kwargs):
    blob_text, urls = fetch_homepage_categories_and_articles(home_url, **kwargs)
    user_prompt = f"""
You are creating a digest for expats based on these scraped pages.
The data below may contain navigation text; focus on actual news content.

Scraped data:
{blob_text}

All article URLs (for reference):
{chr(10).join(urls)}

Remember to include a "Sources" at the end of each section.
"""
    return user_prompt



## Step 4: final digest

In [9]:
def create_expat_digest(home_url, **kwargs):
    prompt = get_digest_user_prompt(home_url, **kwargs)
    response = openai.chat.completions.create(
        model=MODEL_DIGEST,
        messages=[
            {"role": "system", "content": digest_system_prompt},
            {"role": "user", "content": prompt},
        ],
    )
    return display(Markdown(response.choices[0].message.content))

def stream_expat_digest(home_url, **kwargs):
    print("Preparing a report")
    prompt = get_digest_user_prompt(home_url, **kwargs)
    stream = openai.chat.completions.create(
        model=MODEL_DIGEST,
        messages=[
            {"role": "system", "content": digest_system_prompt},
            {"role": "user", "content": prompt},
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)


    

## Step 5: set up gradio

In [10]:
title_name = "Breaking News"
news_source = gr.Textbox(label="News outlet page URL including http:// or https://")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_expat_digest,
    title=title_name,
    inputs=[news_source],
    outputs=[message_output],
    examples=[
        ["https://www.dutchnews.nl/"], ["https://nltimes.nl"],
        ],
    flagging_mode="never"
)

## Try it out

In [ ]:
view.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Preparing a report
Selecting relevant category links for https://nltimes.nl by calling gpt-5-nano
Found 9 relevant category links
Selected 5 articles
Selected 6 articles
Selected 5 articles
Fetching data


# This Week in the Netherlands: Expats News Digest

## What were important events this week
- Amsterdam UMC performed the world's first brain surgery using mixed reality hologram technology.
- Zwolle Court sentenced a leader of a violent human trafficking gang to 20 years in prison.
- Amsterdam Court of Appeal ruled that Uber drivers are freelancers, not employees.
- Smog warning issued for Northern Netherlands due to climbing fine dust levels.
- Four Afghan women were granted asylum after initially being rejected.
- Dutch municipalities are facing a €2 billion funding gap by 2029.
- The number of organ transplants slightly decreased in 2025, despite a long-term upward trend.

---

## Health & Innovation

- **World first: brain surgery using hologram technology**  
  Amsterdam UMC neurosurgeon successfully performed brain surgery with mixed reality glasses projecting a 3D brain hologram onto the patient's head, aiming to improve accuracy for a common procedure.  
  *Why it matters:* This technology could reduce surgical mistakes and improve outcomes, potentially impacting healthcare quality for expats using Dutch hospitals.  
  *What you can do:* Stay informed on healthcare innovations; hospitals may adopt new tech that affects treatment processes.

- **Fewer organ transplants last year, but trend still upward**  
  In 2025, 1,480 organ transplants occurred, 7% fewer than 2024, but a general long-term increase remains due to policy changes encouraging more organ donations.  
  *Why it matters:* Waiting times for transplants like kidneys and hearts are improving, important if you or family members ever find themselves needing such care.  
  *What you can do:* Register your choice in the Dutch organ donor register or confirm your status, as default "no objection" applies since 2021.

- **Dutch charity grants 25,000th final wish to terminally ill patients**  
  Stichting Ambulance Wens helps terminally ill, immobile patients achieve final wishes, marking a milestone with their 25,000th fulfilled request, with trips accompanied by nurses and volunteers.  
  *Why it matters:* Reflects strong community support and care quality for vulnerable people in the Netherlands.  
  *What you can do:* If you know someone in need, recommend this charity or volunteer.

- **RIVM issues smog warning across Northern Netherlands**  
  The National Institute for Public Health issued a smog warning due to rising fine dust levels, with health risks linked to lung function reduction, especially north of the Amsterdam–Arnhem line.  
  *Why it matters:* Poor air quality can affect respiratory health, especially for children, elderly, and those with conditions—expats should be aware when spending time outdoors.  
  *What you can do:* Limit outdoor exposure on smog alert days, consider air purifiers indoors.

**Sources:**  
- https://nltimes.nl/2026/01/27/world-first-amsterdam-umc-surgeon-performs-brain-surgery-using-mixed-reality-hologram  
- https://nltimes.nl/2026/01/26/fewer-organ-transplants-netherlands-last-year-trend-still-increasing  
- https://nltimes.nl/2026/01/25/dutch-charity-grants-25000th-final-wish-terminally-ill-patient  
- https://nltimes.nl/2026/01/25/video-rivm-issues-smog-warning-across-northern-netherlands-fine-dust-levels-climb

---

## Crime & Safety

- **Zwolle Court hands 20-year sentence to Eritrean human trafficking gang leader**  
  A major human trafficking ring led by an Eritrean was dismantled, resulting in a hefty prison sentence.  
  *Why it matters:* Human trafficking is a serious issue in the Netherlands; staying informed means better awareness of potential risks in your community.

- **Amsterdam man arrested for shooting two Syrian teenagers**  
  The suspect was arrested in connection with shootings on New Year’s Day resulting in two deaths.  
  *Why it matters:* Shows that violent crime can happen anywhere, but generally remains low; expats should stay alert in unfamiliar neighborhoods.

- **Police recovered hidden documents linked to a stabbing suspect in Amsterdam**  
  Investigation ongoing into violent crime involving a Ukrainian suspect.  
  *Why it matters:* Highlights ongoing crime safety challenges in urban areas—personal vigilance matters.

- **One in six young Dutch men confronted with child sex abuse imagery**  
  Authorities reveal worrying encounters with illegal content among youth, raising alarms about online safety.  
  *Why it matters:* If you have children or teenagers, awareness about online dangers is crucial to keep them safe.

**Sources:**  
- https://nltimes.nl/2026/01/27/zwolle-court-sentences-eritrean-20-years-leading-violent-human-trafficking-gang  
- https://nltimes.nl/2026/01/27/amsterdam-man-arrested-shooting-deaths-two-syrian-teenagers-new-years-day  
- https://nltimes.nl/2026/01/27/police-recover-hidden-documents-ukrainian-suspect-amsterdam-stabbing-case  
- https://nltimes.nl/2026/01/27/one-six-young-dutch-men-encountered-child-sex-abuse-images

---

## Politics & Society

- **Four Afghan women granted asylum after prior rejections**  
  The government overturned earlier denials, granting asylum to these women presumably under pressure from human rights considerations.  
  *Why it matters:* Shows Dutch asylum policies can change and might favor vulnerable individuals, affecting how expats view refugee integration.

- **Dutch municipalities face €2 billion funding gap by 2029**  
  An accounting firm warns that local governments will struggle financially in the near future, potentially impacting public services.  
  *Why it matters:* Public services like housing, education, and transport might deteriorate or face cuts—a heads-up if you rely on municipal offerings.

- **Most parents want mandatory e-bike helmets; teens disagree**  
  A generational clash over safety gear; while parents push for rules, teens aren't buying it.  
  *Why it matters:* If you're cycling (or have kids who cycle), helmet laws and cultural acceptance might affect daily habits and even insurance coverage.

- **Petition for Dutch boycott of U.S. World Cup passes 150,000 signatures**  
  Let’s just say, not everyone is thrilled with the World Cup being linked to politics.  
  *Why it matters:* Reflects societal divisions on international issues—maybe not crucial unless you’re into football politics.

**Sources:**  
- https://nltimes.nl/2026/01/27/four-afghan-women-granted-asylum-netherlands-earlier-rejections  
- https://nltimes.nl/2026/01/27/dutch-municipalities-face-eu2-billion-funding-gap-2029-accounting-firm-warns  
- https://nltimes.nl/2026/01/27/parents-want-mandatory-helmets-e-bikes-teens-disagree  
- https://nltimes.nl/2026/01/27/petition-urging-dutch-world-cup-boycott-us-passes-150000-signatures

---

## Business & Economy

- **Amsterdam Court rules Uber drivers are freelancers, not employees**  
  The ruling confirms that Uber drivers do not have employee rights but work as freelancers, affecting job security and benefits.  
  *Why it matters:* Gig economy workers among expats should be aware that they likely won’t get typical employee protections.

- **Number of cafés continues to decline as Gen Z prefers coffee bars**  
  Traditional Dutch cafés are closing as younger people shift their drinking habits.  
  *Why it matters:* Social life and local hangouts are evolving—good to know if you’re scouting for a place to chill out.

- **Dutch unicorn Mews valued at €2 billion after fresh funding**  
  The tech scene is booming, showing the Netherlands still punches above its weight globally.  
  *Why it matters:* Opportunities for expats in tech jobs and startups are growing.

**Sources:**  
- https://nltimes.nl/2026/01/27/uber-drivers-are-freelancers-not-employees-amsterdam-court-appeal-rules  
- https://nltimes.nl/2026/01/27/number-cafes-netherlands-continues-decline-gen-z-tends-coffee-bars  
- https://nltimes.nl/2026/01/26/dutch-unicorn-tech-firm-mews-gets-2-billion-valuation-after-new-funding-round

---

If you want to keep your finger on the pulse without drowning in news, bookmark this digest and check back next week. The Netherlands may be small, but it’s never short of stories that matter to you.

Preparing a report
Selecting relevant category links for https://www.dutchnews.nl/ by calling gpt-5-nano
Found 9 relevant category links
Selected 5 articles
Selected 6 articles
Selected 6 articles
Fetching data


# Weekly News Digest for Expats in The Netherlands  
*Date: Week ending January 27, 2026*

## What were important events this week?  
- Dutch homeowners claimed a whopping €25 billion in mortgage-related tax deductions in 2024, an increase despite political calls to phase out this generosity.  
- The Dutch economy is losing its competitive edge, with concerns raised about monopolies, stifled innovation, and the growing power of big corporations.  
- Flower and plant exports reached €7.2 billion in 2025, close to record levels, but new US tariffs threaten this vital sector.  

---

## Economy & Taxes  

### Homeowners Deductions Still Costing the Treasury Big Time  
Dutch homeowners deducted nearly €25 billion from their taxable income in 2024 through mortgage interest relief and rebates, a 7% rise on 2023. The biggest winners? Families with kids and high earners. Despite decades of criticism and political promises, this tax break remains stubbornly high.  
**Why it matters:** If you’re planning on buying a house, this tax relief might influence your calculations — but brace yourself, it’s on the political chopping block, so don’t bank on it forever.  
**What you can do:** Keep an eye on government announcements if you’re a homeowner or prospective buyer. Planning your finances without overreliance on this deduction is smart.  

### Dutch Economy is Losing Its Mojo  
The competition watchdog warns that the Dutch economy has been getting less competitive over the last decade, especially in tech and pharmaceuticals — thanks in part to large companies snapping up potential rivals to kill competition before it even starts (thanks, “killer acquisitions”). This consolidation isn't just bad news for prices; it could stifle innovation and market resilience.  
**Why it matters:** Less competition often means higher prices and fewer choices, plus slower tech breakthroughs — all not great if you’re an expat trying to make sense of the local economy and job markets.  
**What you can do:** Support small and innovative businesses where you can, and watch out for signs of monopolistic practices in your sectors of interest.  

### Flower and Plant Exports Still Blooming, Despite Tariff Storm  
Dutch flower exports hit €7.2 billion in 2025, with the US market growing until new tariffs took effect late in the year. Germany and the UK remain key customers, though their demand is shrinking slightly.  
**Why it matters:** This is a reminder that global politics and trade policies can hit Dutch staples hard, potentially affecting jobs and prices in the horticulture sector, which is a big employer in parts of the country.  
**What you can do:** If you work in or buy from this sector, stay informed about tariff changes and look for alternative markets or suppliers if the US becomes less accessible.  

---

## Sources  
- https://www.dutchnews.nl/2026/01/home-owners-deductions-cut-e25bn-from-income-tax-bills-in-2024/  
- https://www.dutchnews.nl/2026/01/dutch-economy-is-becoming-less-competitive-watchdog-warns/  
- https://www.dutchnews.nl/2026/01/flower-and-plant-exports-top-e7-billion-tariffs-hit-us-market/  

---

**That’s your mid-winter Dutch update with a pinch of reality: The economy’s getting a bit weird, the housing perks might not stick around forever, and even tulips and roses aren't immune to trade wars. Stay savvy, and don’t let the politics or market quirks catch you off guard!**